In [1]:
### Import tensorflow, keras, and numpy
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time

2021-08-09 13:07:27.947032: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-08-09 13:07:27.947068: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
### Import required objects from keras
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, BatchNormalization, MaxPool2D
from keras.layers import Activation, Input, Flatten, Dropout
from keras.models import Model
from keras.utils import to_categorical

In [3]:
### Set Random Seed
seed = np.random.default_rng(2021)

### Load in the dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

### Number of classes
print(f'The dataset has {len(np.unique(y_train))} distinct classes.')

### Scale down the datasets
X_train, X_test = X_train.astype('float32')/255, X_test.astype('float32')/255

### Network params
IMAGE_SIZE = X_train.shape[1]
BATCH_SIZE = 64
INPUT_SHAPE = [IMAGE_SIZE, IMAGE_SIZE, 1] ### Image width, image height, number of channels

11493376/11490434 [==============================] - 0s 0us/step
The dataset has 10 distinct classes.


In [4]:
IMAGE_SIZE

28

In [5]:
INPUT_SHAPE

[28, 28, 1]

In [6]:
X_train.shape

(60000, 28, 28)

In [7]:
y_train.shape

(60000,)

In [8]:
y_test.shape

In [9]:
### Reshape and one-hot encode class labels
y_train = np.reshape(y_train, newshape = (60000, 1))
y_test = np.reshape(y_test, newshape = (10000, 1))

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
### Build CNN classifier
def build_cnn_classifier():
    inputs = Input(shape = INPUT_SHAPE, name = 'Input layer')
    ### 28 BY 28
    x = Conv2D(filters = 6, kernel_size = 3, padding = 'same') (inputs)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)
    ### 26 BY 26
    x = Conv2D(filters = 16, kernel_size = 3, padding = 'same', activation = 'relu') (x)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)
    ### 24 BY 24
    x = Conv2D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu') (x)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)
    ### 22 BY 22
    x = Conv2D(filters = 64, kernel_size = 3, padding = 'same', activation = 'relu') (x)
    x = MaxPool2D(pool_size = 2, padding = 'same') (x)

    ### Flatten
    x = Flatten() (x)
    x = Dropout(0.2) (x)
    x = Dense(128, activation = 'relu') (x)
    x = Dropout(0.2) (x)
    x = Dense(64, activation = 'relu') (x)
    x = Dropout(0.2) (x)
    x = Dense(10, activation = 'softmax') (x)

    return Model(inputs = inputs, outputs = x)

In [11]:
cnn_model = build_cnn_classifier()

2021-08-09 13:07:34.469681: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-08-09 13:07:34.472662: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-09 13:07:34.472687: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-09 13:07:34.472712: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-cedde139-93f2-4278-8e9d-d80931c4b7dc): /proc/driver/nvidia/version does not exist
2021-08-09 13:07:34.473571: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operatio

In [12]:
### Encoder summary
cnn_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input layer (InputLayer)     [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 6)         60        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        880       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 32)          4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 32)          0     

In [13]:
### Import and instantiate optimizer
from keras.optimizers import RMSprop, SGD
optimizer = SGD(learning_rate = 0.001, momentum = 0.0)

### Compile model
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = optimizer,
                  metrics = ['accuracy'])

In [14]:
### Import and instantiate a learning rate scheduler
from keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(monitor = 'val_loss', patience = 5,
                                 verbose = 1, min_lr = 0.0001, factor = 0.1)

In [15]:
### Fit model
cnn_model.fit(X_train, y_train, epochs = 100, batch_size = BATCH_SIZE, verbose = 2,
              callbacks = [lr_scheduler], validation_data = (X_test, y_test))

2021-08-09 13:07:34.945201: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-08-09 13:07:34.956599: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2500000000 Hz
Epoch 1/100
938/938 - 36s - loss: 2.2961 - accuracy: 0.1174 - val_loss: 2.2815 - val_accuracy: 0.1466
Epoch 2/100
938/938 - 45s - loss: 2.2799 - accuracy: 0.1563 - val_loss: 2.2565 - val_accuracy: 0.2785
Epoch 3/100
938/938 - 39s - loss: 2.2558 - accuracy: 0.2007 - val_loss: 2.2141 - val_accuracy: 0.3402
Epoch 4/100
938/938 - 33s - loss: 2.2063 - accuracy: 0.2503 - val_loss: 2.1165 - val_accuracy: 0.4419
Epoch 5/100
938/938 - 32s - loss: 2.0836 - accuracy: 0.3043 - val_loss: 1.8514 - val_accuracy: 0.5700
Epoch 6/100
938/938 - 32s - loss: 1.8221 - accuracy: 0.3836 - val_loss: 1.3975 - val_accuracy: 0.6598
Epoch 7/100
938/938 - 32s - loss: 1.4541 - accuracy: 0.5092 - val_loss: 0.9264 - val_accuracy: 0.7688
Epoch 8/100
93

In [16]:
### Check model performance
train_loss, train_acc = cnn_model.evaluate(X_train, y_train, verbose = 1)
test_loss, test_acc = cnn_model.evaluate(X_test, y_test, verbose = 1)

313/313 [==============================] - 2s 7ms/step - loss: 0.0561 - accuracy: 0.9818


In [17]:
### Loss on train dataset
train_loss

0.05420488864183426

In [18]:
### Accuracy on train dataset
train_acc

0.983299970626831

In [19]:
### Loss on test dataset
test_loss

0.05607939884066582

In [20]:
### Accuracy on test dataset
test_acc

0.9818000197410583

In [21]:
from datetime import datetime
instant = datetime.strftime(datetime.now(), '%d%a-%H:%M')

In [22]:
instant

'09Mon-14:02'

In [24]:
### Save model weights in .h5 format
cnn_model.save_weights(filepath = '/work/MNIST_model_weights_{}.h5'.format(datetime.strftime(datetime.now(), '%d%a-%H:%M')),
                       save_format = 'hdf5')

In [25]:
### Save model weights in .hdf5 format
cnn_model.save_weights(filepath = '/work/MNIST_model_weights_{}.hdf5'.format(datetime.strftime(datetime.now(), '%d%a-%H:%M')),
                       save_format = 'hdf5')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=cedde139-93f2-4278-8e9d-d80931c4b7dc' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>